<font size="8"> **Adding environmental data from ACCESS-OM2-01 to unique crabeater observations** </font>  
In this notebook, we will extract environmental data from the ACCESS-OM2-01 model outputs and add it to our data frame containing unique crabeater sightings per month and grid cell (see `04_Creating_bio_data_masks.ipynb` for more information.

# Setting working directory
In order to ensure these notebooks work correctly, we will set the working directory. We assume that you have saved a copy of this repository in your home directory (represented by `~` in the code chunk below). If you have saved this repository elsewhere in your machine, you need to ensure you update this line with the correct filepath where you saved these notebooks.

In [1]:
import os
os.chdir(os.path.expanduser('~/Chapter2_Crabeaters/Scripts'))

# Loading other relevant libraries

In [2]:
from dask.distributed import Client
from glob import glob
#Accessing model data
import cosima_cookbook as cc
#Useful functions
import UsefulFunctions as uf
#Dealing with data
import xarray as xr
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Paralellising work 

In [3]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 4
Total threads: 12,Total memory: 48.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37615,Workers: 4
Dashboard: /proxy/8787/status,Total threads: 12
Started: Just now,Total memory: 48.00 GiB
Comm: tcp://127.0.0.1:34895,Total threads: 3
Dashboard: /proxy/34655/status,Memory: 12.00 GiB
Nanny: tcp://127.0.0.1:46723,


# Loading unique crabeater seal observations data frame

In [30]:
#Loading dataset as pandas data frame
crabeaters = pd.read_csv('../Cleaned_Data/unique_crabeater_obs_grid.csv')

#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)

#Checking results
crabeaters

,event_date,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,decade,sector,zone,presence
0,2019-03-09T00:00:00Z,2019-03,2019,3,-64.888,-62.85,-64.910,-62.9,autumn,in-between,2010,East Pacific,Antarctic,1
1,2019-03-13T00:00:00Z,2019-03,2019,3,-65.142,-64.15,-65.121,-64.1,autumn,in-between,2010,East Pacific,Antarctic,1
2,2019-03-15T00:00:00Z,2019-03,2019,3,-65.142,-64.05,-65.121,-64.1,autumn,in-between,2010,East Pacific,Antarctic,1
3,2019-03-15T00:00:00Z,2019-03,2019,3,-65.100,-63.95,-65.079,-64.0,autumn,in-between,2010,East Pacific,Antarctic,1
4,2019-03-14T00:00:00Z,2019-03,2019,3,-65.353,-64.15,-65.332,-64.1,autumn,in-between,2010,East Pacific,Antarctic,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3975,2006-02-19T13:00:00Z,2006-02,2006,2,-67.085,69.95,-67.064,70.0,summer,moulting,2000,Central Indian,Antarctic,1
3976,2006-01-30T13:00:00Z,2006-01,2006,1,-67.972,40.75,-67.951,40.7,summer,moulting,2000,Central Indian,Antarctic,1
3977,2006-02-19T13:00:00Z,2006-02,2006,2,-67.085,69.95,-67.106,70.0,summer,moulting,2000,Central Indian,Antarctic,1
3978,2006-02-19T13:00:00Z,2006-02,2006,2,-67.127,69.95,-67.149,70.0,summer,moulting,2000,Central Indian,Antarctic,1


# Loading crabeater seal masks

In [4]:
mask_all = xr.open_dataarray('/g/data/v45/la6889/Chapter2_Crabeaters/mask_crabeater_obs_ocean_grid.nc')
mask_vel = xr.open_dataarray('/g/data/v45/la6889/Chapter2_Crabeaters/mask_crabeater_obs_vel_grid.nc')

# Adding values for static variables only
Static variables referred to any physical variables that do not change over time (at least not during the time period of our interest). Examples include depth of the water column and distance to coastline. Given that we only have one value for these variables, the process of extracting data is relatively simple. We do not need to take into account the date observations were collected.

## Defining dictionary with information about static variables
This dictionary contains the column labels for each and the name of the files for each static variable to be included in our analysis. We will also define a variable containing the full path to the folder where all static variables are stored.

In [8]:
#Full path to static variables
base_dir_static = '/g/data/v45/la6889/Chapter2_Crabeaters/Static_Variables/'

#List of static variables
varDict = {'bottom_slope_deg': 'bathy_slope_GEBCO_2D.nc',
           'dist_shelf_km': 'distance_shelf.nc',
           'dist_coast_km': 'distance_coastline.nc',
           'depth_m': 'bathy_GEBCO_2D.nc'}

## Extracting data for each observation and adding it to a new column in crabeater data

In [31]:
#Looping through dictionary keys
for var in varDict:
    #Creating full path to file of interest
    file_path = os.path.join(base_dir_static, varDict[var])
    #Load as raster
    ras = xr.open_dataarray(file_path)
    #Applying mask
    ras_masked = ras*mask_all
    #Transforming masked array into data frame
    ras_df = ras_masked.to_series().dropna().reset_index()
    #Rounding up coordinate values
    ras_df = ras_df.round({'yt_ocean': 3, 'xt_ocean': 3})
    #Renaming masked data before merging to observations
    ras_df.rename(columns = {0: var}, inplace = True)
    #Adding to crabeater observations data frame
    crabeaters = crabeaters.merge(ras_df, on = ['yt_ocean', 'xt_ocean'], how = 'left').sort_values(['yt_ocean', 'xt_ocean'])
    
#Checking results
crabeaters

,event_date,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,decade,sector,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,2004-02-19T00:00:00Z,2004-02,2004,2,-77.687,166.35,-77.708,166.3,summer,moulting,2000,West Pacific,Antarctic,1,89.980469,606.013921,32.155391,176.033340
1,2014-01-13T14:51:22Z,2014-01,2014,1,-76.842,-36.05,-76.864,-36.1,summer,moulting,2010,Atlantic,Antarctic,1,89.729942,383.202891,41.998688,1064.320435
2,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-157.25,-76.779,-157.3,summer,moulting,2000,West Pacific,Antarctic,1,89.943176,113.770361,4.724272,612.181458
3,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-156.85,-76.779,-156.8,summer,moulting,2000,West Pacific,Antarctic,1,89.897667,113.278688,2.641858,656.099976
4,2020-02-02T00:00:00Z,2020-02,2020,2,-76.378,163.95,-76.357,163.9,summer,moulting,2020,West Pacific,Antarctic,1,89.871246,603.310632,21.026712,519.266663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3975,1989-11-05T10:00:00Z,1989-11,1989,11,-60.047,70.25,-60.072,70.2,autumn,weaning,1980,Central Indian,Subantarctic,1,89.978432,478.561536,622.374268,4235.086426
3976,2008-12-26T00:00:00Z,2008-12,2008,12,-59.645,9.15,-59.620,9.1,summer,weaning,2000,Atlantic,Antarctic,1,89.958000,844.112672,926.684712,5462.944336
3977,1989-11-05T02:00:00Z,1989-11,1989,11,-59.493,71.85,-59.468,71.9,autumn,weaning,1980,Central Indian,Subantarctic,1,89.864761,531.076147,692.923480,4498.345215
3978,2013-11-12T00:00:00Z,2013-11,2013,11,-57.829,-26.35,-57.803,-26.3,autumn,weaning,2010,Atlantic,Antarctic,1,NaN,1443.071239,1067.076760,506.524384


## Saving data frame with static variables
Given that the dynamic variables take some time to extract. We will save intermediary results to avoid having to extract them again.

In [6]:
crabeaters.to_csv('../Cleaned_Data/unique_crabeater_obs_static_env.csv', index = False)

# Adding values for dynamic variables
Given the amount of crabeater seal observations and the time period covered by this dataset, the extraction of these values may take some time. It is recommended to save the data frame after every time a new variable is extracted. This way we can avoid losing data.

In [8]:
crabeaters = pd.read_csv('../Cleaned_Data/unique_crabeater_obs_static_env.csv')
#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)
crabeaters

,event_date,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,decade,sector,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,2004-02-19T00:00:00Z,2004-02,2004,2,-77.687,166.35,-77.708,166.3,summer,moulting,2000,West Pacific,Antarctic,1,89.980469,606.013921,32.155391,176.033340
1,2014-01-13T14:51:22Z,2014-01,2014,1,-76.842,-36.05,-76.864,-36.1,summer,moulting,2010,Atlantic,Antarctic,1,89.729942,383.202891,41.998688,1064.320435
2,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-157.25,-76.779,-157.3,summer,moulting,2000,West Pacific,Antarctic,1,89.943176,113.770361,4.724272,612.181458
3,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-156.85,-76.779,-156.8,summer,moulting,2000,West Pacific,Antarctic,1,89.897667,113.278688,2.641858,656.099976
4,2020-02-02T00:00:00Z,2020-02,2020,2,-76.378,163.95,-76.357,163.9,summer,moulting,2020,West Pacific,Antarctic,1,89.871246,603.310632,21.026712,519.266663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3975,1989-11-05T10:00:00Z,1989-11,1989,11,-60.047,70.25,-60.072,70.2,autumn,weaning,1980,Central Indian,Subantarctic,1,89.978432,478.561536,622.374268,4235.086426
3976,2008-12-26T00:00:00Z,2008-12,2008,12,-59.645,9.15,-59.620,9.1,summer,weaning,2000,Atlantic,Antarctic,1,89.958000,844.112672,926.684712,5462.944336
3977,1989-11-05T02:00:00Z,1989-11,1989,11,-59.493,71.85,-59.468,71.9,autumn,weaning,1980,Central Indian,Subantarctic,1,89.864761,531.076147,692.923480,4498.345215
3978,2013-11-12T00:00:00Z,2013-11,2013,11,-57.829,-26.35,-57.803,-26.3,autumn,weaning,2010,Atlantic,Antarctic,1,NaN,1443.071239,1067.076760,506.524384


## Accessing ACCESS-OM2-01 model outputs
We will create a new `cosima cookbook` session to load the model outputs of interest, and we will also create a dictionary that contains useful information related to data extraction.

In [5]:
#Creating new COSIMA cookbook session
session = cc.database.create_session()

#Creating dictionary with useful information
varDict = {'model': 'ACCESS-OM2-01',
           #ACCESS-OM2-01 cycle 4 (1958-2018)
           'exp': '01deg_jra55v140_iaf_cycle4',
           #ACCESS-OM2-01 cycle 4 extension (2018-2022)
           'exp_ext': '01deg_jra55v140_iaf_cycle4_jra55v150_extension',
           #Temporal resolution
           'freq': '1 monthly',
           #Output folder
           'base_out': '../Cleaned_Data'}

## Loading data frame with ACCESS-OM2-01 outputs
We can use this data frame to find the variable names for the environmental factors that we know are influential for the distribution of crabeater seals.

In [102]:
#Loading data frame with model outputs
# var_acc = cc.querying.get_variables(session, experiment = varDict['exp_ext'], frequency = '1 monthly')

#Searching data frame for variables of interest
var_acc[var_acc.name.str.contains('salt')]

,name,long_name,units,frequency,ncfile,cell_methods,# ncfiles,time_start,time_end
58,fsalt_ai_m,salt flux ice to ocean,kg/m^2/s,1 monthly,output1008/ice/OUTPUT/iceh.2023-03.nc,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
59,fsalt_m,salt flux ice to ocn (cpl),kg/m^2/s,1 monthly,output1008/ice/OUTPUT/iceh.2023-03.nc,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
96,salt,Practical Salinity,psu,1 monthly,output1008/ocean/ocean-3d-salt-1-monthly-mean-...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
97,salt_xflux_adv,rho*dzt*dyt*u*tracer,kg/sec,1 monthly,output1008/ocean/ocean-3d-salt_xflux_adv-1-mon...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
98,salt_yflux_adv,rho*dzt*dxt*v*tracer,kg/sec,1 monthly,output1008/ocean/ocean-3d-salt_yflux_adv-1-mon...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
105,sfc_salt_flux_coupler,sfc_salt_flux_coupler: flux from the coupler,kg/(m^2*sec),1 monthly,output1008/ocean/ocean-2d-sfc_salt_flux_couple...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
106,sfc_salt_flux_ice,sfc_salt_flux_ice,kg/(m^2*sec),1 monthly,output1008/ocean/ocean-2d-sfc_salt_flux_ice-1-...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
107,sfc_salt_flux_restore,sfc_salt_flux_restore: flux from restoring term,kg/(m^2*sec),1 monthly,output1008/ocean/ocean-2d-sfc_salt_flux_restor...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00
120,surface_salt,Practical Salinity,psu,1 monthly,output1008/ocean/ocean-2d-surface_salt-1-month...,time: mean,51,2019-01-01 00:00:00,2023-04-01 00:00:00


## Completing dictionary with useful variables
Now that we identified the correct name for the variable of our interest, we can complete our dictionary.

In [72]:
#Variable name in the model
varDict['var_mod'] = 'aice_m'
#Name of column where we will store the extracted data
varDict['var_short_name'] = 'lt_pack_ice'
#Defining if this variable is related to sea ice or not
varDict['ice_data'] = False
#Checking final dictionary
varDict

{'model': 'ACCESS-OM2-01',
 'exp': '01deg_jra55v140_iaf_cycle4',
 'exp_ext': '01deg_jra55v140_iaf_cycle4_jra55v150_extension',
 'freq': '1 monthly',
 'base_out': '../Cleaned_Data',
 'var_mod': 'aice_m',
 'var_short_name': 'lt_pack_ice',
 'ice_data': False}

## Loading data from ACCESS-OM2-01
The crabeater data has data between 1978 and 2022. The ACCESS-OM2-01 originally had a temporal range of 60 years (1958-2018). However, for the 4th model run (used in these notebooks), the model was extended to December 2022. These outputs are available in two different experiments, which are loaded and merged below before data extraction occurs.

In [50]:
#Loading data from fourth cycle (temporal range 1958 to 2018)
var_df = uf.getACCESSdata_SO(varDict['var_mod'], '1978-01', '2019-01', 
                              freq = varDict['freq'], ses = session, minlat = -80,
                              exp = varDict['exp'], ice_data = varDict['ice_data'])
#Loading data from fourth cycle extension (2019 to 2022)
var_df_ext = uf.getACCESSdata_SO(varDict['var_mod'], '2019-01', '2023-01', 
                              freq = varDict['freq'], ses = session, minlat = -80,
                              exp = varDict['exp_ext'], ice_data = varDict['ice_data'])

#Concatenating both data arrays into one
var_df = xr.concat([var_df, var_df_ext], dim = 'time')

#Transforming longitudes so their range is +/-180 degrees
var_df = uf.corrlong(var_df)

#Removing duplicate variable
del var_df_ext

## *Optional: Data transformation*
Some variables, such as sea surface temperature are in units of Kelvin instead of $^{\circ}C$ (shown below). In this section, we transform variables to the units of our choice.

In [56]:
#Temperature data is transformed from Kelvins to C
var_df = var_df-273.15

## *Optional: Subsetting surface layer data*
For some ocean variables, we need to subset data to extract surface values or bottom values. Subsetting data for the surface layer is an easy process, we simply need to select the first depth bin available. The `st_ocean` dimension contains the depth bins.

In [42]:
#Selecting the first depth available in the model (i.e. surface layer)
var_df = var_df.isel(st_ocean = 0)

#Checking results - dataset has three dimensions instead of the original four
var_df

<xarray.DataArray 'u' (time: 541, yu_ocean: 714, xu_ocean: 3600)>
dask.array<getitem, shape=(541, 714, 3600), dtype=float32, chunksize=(1, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 -180.0 -179.9 -179.8 ... 179.7 179.8 179.9
  * yu_ocean  (yu_ocean) float64 -79.99 -79.95 -79.9 ... -45.14 -45.07 -45.0
    st_ocean  float64 0.5413
  * time      (time) datetime64[ns] 1977-12-16T12:00:00 ... 2022-12-16T12:00:00
Attributes: (12/13)
    long_name:      i-current
    units:          m/sec
    valid_range:    [-10.  10.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_c geolat_c
    ...             ...
    ncfiles:        ['/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_...
    contact:        Andrew Kiss
    email:          andrew.kiss@anu.edu.au
    created:        2022-04-27
    description:    0.1 degree ACCESS-OM2 global model configuration under in...
    notes:          Run configuration and history: https://github.com/COSIMA/...

## *Optional: Subsetting bottom data*
Subsetting data for the bottom layer is not as straightforward as for the surface. This is because the bathymetry is not the same across the Southern Ocean. To identify the correct depth bin that contains the deepest values for a particular grid cell.

In [51]:
mask_2d = xr.where(~np.isnan(var_df.isel(time = 0)), 1, np.nan)
mask_2d = mask_2d.cumsum('st_ocean').where(~np.isnan(var_df.isel(time = 0)))
mask_2d = xr.where(mask_2d == mask_2d.max('st_ocean'), 1, np.nan)
mask_2d

<xarray.DataArray 'u' (st_ocean: 75, yu_ocean: 714, xu_ocean: 3600)>
dask.array<where, shape=(75, 714, 3600), dtype=float64, chunksize=(19, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 -180.0 -179.9 -179.8 ... 179.7 179.8 179.9
  * yu_ocean  (yu_ocean) float64 -79.99 -79.95 -79.9 ... -45.14 -45.07 -45.0
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
    time      datetime64[ns] 1977-12-16T12:00:00

In [52]:
var_2d = (mask_2d*var_df).sum('st_ocean')
var_2d

<xarray.DataArray 'u' (yu_ocean: 714, xu_ocean: 3600, time: 541)>
dask.array<sum-aggregate, shape=(714, 3600, 541), dtype=float64, chunksize=(135, 180, 1), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 -180.0 -179.9 -179.8 ... 179.7 179.8 179.9
  * yu_ocean  (yu_ocean) float64 -79.99 -79.95 -79.9 ... -45.14 -45.07 -45.0
  * time      (time) datetime64[ns] 1977-12-16T12:00:00 ... 2022-12-16T12:00:00

In [53]:
if var_df.name in ['u', 'v']:
    var_df = var_2d.transpose('time', 'yu_ocean', 'xu_ocean')
else:
    var_df = var_2d.transpose('time', 'yt_ocean', 'xt_ocean')
#Checking results
var_df

<xarray.DataArray 'u' (time: 541, yu_ocean: 714, xu_ocean: 3600)>
dask.array<transpose, shape=(541, 714, 3600), dtype=float64, chunksize=(1, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * xu_ocean  (xu_ocean) float64 -180.0 -179.9 -179.8 ... 179.7 179.8 179.9
  * yu_ocean  (yu_ocean) float64 -79.99 -79.95 -79.9 ... -45.14 -45.07 -45.0
  * time      (time) datetime64[ns] 1977-12-16T12:00:00 ... 2022-12-16T12:00:00

## Loading dynamic variables derived from calculations
These variables are not directly available in ACCESS-OM2-01, but they have been calculated from model outputs. Refer to folder `02_Environmental_Data` to see the full details of each calculation.

In [71]:
var_df = xr.open_mfdataset(sorted(glob('/g/data/v45/la6889/Chapter2_Crabeaters/SeaIce/LongTerm_PackIce/*.nc'))).aice_m
var_df

<xarray.DataArray 'aice_m' (time: 540, yt_ocean: 713, xt_ocean: 3600)>
dask.array<concatenate, shape=(540, 713, 3600), dtype=float64, chunksize=(12, 713, 3600), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
  * yt_ocean  (yt_ocean) float64 -79.97 -79.93 -79.88 ... -45.18 -45.11 -45.04
  * time      (time) datetime64[ns] 1978-01-01 1978-02-01 ... 2022-12-01

## Subsetting variables by time to match observations
By the subsetting the original dataset, we will reduce computing time.

In [73]:
#Creating a list with the year and month when crabeaters were observed - Based on mask time steps
timestep = sorted(crabeaters.date.unique())

#Creating an empty list to keep the subset model data
var_df_int = []
#Looping through each time
for t in timestep:
    var_df_int.append(var_df.sel(time = t))
    
#Creating a new data array with the time steps of interest
var_df_int = xr.concat(var_df_int, dim = 'time')
#Dealing original model data
del var_df

#Checking results - Time steps now match mask
var_df_int

<xarray.DataArray 'aice_m' (time: 140, yt_ocean: 713, xt_ocean: 3600)>
dask.array<concatenate, shape=(140, 713, 3600), dtype=float64, chunksize=(1, 713, 3600), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
  * yt_ocean  (yt_ocean) float64 -79.97 -79.93 -79.88 ... -45.18 -45.11 -45.04
  * time      (time) datetime64[ns] 1978-01-01 1981-12-01 ... 2022-12-01

## Applying crabeater observations mask

In [74]:
#Identifying correct mask to apply
if 'xt_ocean' in var_df_int.coords:
    var_masked = var_df_int.where(~np.isnan(mask_all), drop = True)
elif 'xu_ocean' in var_df_int.coords:
    var_masked = var_df_int.where(~np.isnan(mask_vel), drop = True)

In [75]:
#Converting to pandas data frame
var_pd = var_masked.to_series().dropna().reset_index()

#Adding year and month column prior to merging with crabeater observations
var_pd['year'] = var_pd.apply(lambda i: i.time.year, axis = 1)
var_pd['month'] = var_pd.apply(lambda i: i.time.month, axis = 1)

#Finding name of columns to round up
round_cols = [i for i in var_pd.columns if 'ocean' in i]
#Rounding coordinate values prior to merging
var_pd = var_pd.round({round_cols[0]: 3, round_cols[1]: 3})
#Removing time column that is not needed
var_pd = var_pd.drop(columns = 'time')

#Checking results
var_pd.head()

,yt_ocean,xt_ocean,aice_m,year,month
0,-77.687,166.35,0.702381,1978,1
1,-76.842,-36.05,0.690476,1978,1
2,-76.758,-157.25,0.702381,1978,1
3,-76.758,-156.85,0.690476,1978,1
4,-76.378,163.95,0.821429,1978,1


In [76]:
#Getting column names for merging
cols = var_pd.drop(columns = varDict['var_mod']).columns.tolist()
cols

['yt_ocean', 'xt_ocean', 'year', 'month']

In [77]:
crabeaters = crabeaters.merge(var_pd, on = cols, how = 'left')
crabeaters.rename(columns = {varDict['var_mod']: varDict['var_short_name']}, inplace = True)
crabeaters

,event_date,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,...,SST_degC,bottom_temp_degC,SSS_PSU,bottom_sal_PSU,vel_lat_surf_msec,vel_lat_bottom_msec,vel_lon_surf_msec,vel_lon_bottom_msec,dist_ice_edge_km,lt_pack_ice
0,2004-02-19T00:00:00Z,2004-02,2004,2,-77.687,166.35,-77.708,166.3,summer,moulting,...,-0.398438,-1.389679,33.180775,34.552128,0.070645,-0.009020,-0.045610,0.002839,775.634642,0.702381
1,2014-01-13T14:51:22Z,2014-01,2014,1,-76.842,-36.05,-76.864,-36.1,summer,moulting,...,2.801361,-1.864105,34.046490,34.569469,-0.008134,-0.008721,0.013139,-0.004004,1280.861181,0.690476
2,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-157.25,-76.779,-157.3,summer,moulting,...,-0.370117,1.146729,33.410400,34.672157,-0.047662,-0.001061,-0.025336,-0.001699,445.928328,0.714286
3,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-156.85,-76.779,-156.8,summer,moulting,...,-0.404816,1.145599,33.329450,34.684376,-0.016419,-0.000588,-0.043132,-0.006419,454.552151,0.714286
4,2020-02-02T00:00:00Z,2020-02,2020,2,-76.378,163.95,-76.357,163.9,summer,moulting,...,1.423065,-1.873322,34.118916,34.834370,0.208273,0.032141,-0.074038,0.001561,666.945871,0.785714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3975,1989-11-05T10:00:00Z,1989-11,1989,11,-60.047,70.25,-60.072,70.2,autumn,weaning,...,-1.193298,-0.517700,33.778362,34.633255,-0.053135,-0.001394,0.109026,-0.001630,76.258827,0.000000
3976,2008-12-26T00:00:00Z,2008-12,2008,12,-59.645,9.15,-59.620,9.1,summer,weaning,...,-1.436432,-0.770233,33.698380,34.619843,0.048330,0.015275,0.077441,0.003605,291.618185,0.321429
3977,1989-11-05T02:00:00Z,1989-11,1989,11,-59.493,71.85,-59.468,71.9,autumn,weaning,...,-1.240387,-0.523804,33.757450,34.632874,-0.040124,-0.004590,0.063685,-0.002233,55.971258,0.000000
3978,2013-11-12T00:00:00Z,2013-11,2013,11,-57.829,-26.35,-57.803,-26.3,autumn,weaning,...,-1.099792,0.045715,34.138268,34.659763,0.075677,0.016777,0.062335,0.009469,129.334694,0.000000


In [78]:
crabeaters.to_csv('../Cleaned_Data/unique_crabeater_obs_all_env.csv', index = False)